# Collaborative Filtering Project
## by Brandon Hobson

#### Instructions:
- you can change the number of neighbors on line 79
- the code will loop over all values in K and output Mean Squared Error for all of them
- on lines 19-20 and 82-83, you can change the dataset

In [1]:
from collections import defaultdict
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import numpy as np
import math

# string variable to format ouput easier
output = 'Mean Squared Error'
output += '\nK\tAverage Case\tIgnored Case'

# STORE SIMILARITIES IN A DICTIONARY SIM (THIS IS LIKE A SPARSE-MATRIX WHERE WE ONLY STORE NON-ZERO VALUES)
SIM = defaultdict(dict)
# STORE RATINGS in ITM
ITM = defaultdict(dict)

# ITM[m][u] stores rating score for movie m and user u
# SIM[m1][m2] stores similarity score between movie m and m1

# ifile = open("exampledataset/data.txt")
ifile = open("netflix-small/ratings-train.txt")
for l in ifile:
    parts = l.strip().split(",")
    ITM[int(parts[0])][int(parts[1])] = float(parts[2])
ifile.close()

# dict to hold average ratings for movie from all users
RAT = defaultdict(dict)

# loops to go through each pair of movies
for m in ITM.keys():
    for m1 in ITM.keys():
        if m==m1:
            continue
        # COMPUTE SIMILARITY BETWEEN m and m1 and store this value in SIM[m][m1]
        
        # initialize set of users that rated both movies m and m1
        U = []
        
        # loop to go through movie m's user's who have rated this movie
        for user in ITM[m].keys():
            # user that rated both movies m and m1
            if user in ITM[m1].keys():
                U.append(user)
                
        # since there are no users that have rated both movies we can't assign any similarities to them
        
        if len(U) == 0:
            SIM[m][m1] = 0
            
        if len(U) != 0:
            # average rating for movie m & m1 from all users
            mSum = 0
            for user in ITM[m].keys():
                mSum += ITM[m][user]
            mAvgRating = round(mSum / (len(ITM[m].keys())), 2)
            RAT[m] = mAvgRating
            
            m1Sum = 0
            for user in ITM[m1].keys():
                m1Sum += ITM[m1][user]
            m1AvgRating = round(m1Sum / (len(ITM[m1].keys())), 2)
            RAT[m1] = m1AvgRating

            # calculating the weight / similarity SIM[m][m1]

            num = 0 # numerator of correlation equation
            den1 = 0 # denominator left side of correlation equation
            den2 = 0 # denominator right side of correlation equation
            for user in U:
                num += ((ITM[m][user] - mAvgRating) * (ITM[m1][user] - m1AvgRating))
                den1 += ((ITM[m][user] - mAvgRating) ** 2)
                den2 += ((ITM[m1][user] - m1AvgRating) ** 2)

            den = math.sqrt(den1 * den2)
            result = round((num / den),2)
            SIM[m][m1] = result

# k neighbors to go through
K = [1,2,3,4,5,5,6,7,8,9,10]

for k in K:
    # ifile = open("exampledataset/predictions.txt")
    ifile = open("netflix-small/ratings-test.txt")
    trueRatings = []
    
    # pred ratings using average movie rating for cases where 
    # you can't predict a movie
    predRatings = []
    trueRatings = []
    
    # pred ratings where we ignore cases when you can't predict a movie
    predRatings2 = []
    trueRatings2 = []
    for l in ifile:
        parts = l.strip().split(",")
        movie = int(parts[0])
        user = int(parts[1])
        truerating = float(parts[2])

        trueRatings.append(truerating)

        # FIND K NEIGHBORS for movie from the weights stored in SIM

        # neighbors list
        neighbors = []

        # count all similar items of movie that are rated by user
        n = 0
        for m1 in SIM[movie].keys():
            if user in ITM[m1].keys():
                n +=1

        # loop for each neighbor
        # n is the similar movies that are rated by the user
        # if n is < k only assign n neighbors
        for i in range(min(n,k)):
            best = -1
            bestW = 0
            for m1 in SIM[movie].keys():
                if m1 not in neighbors and user in ITM[m1].keys(): 
                    weight = SIM[movie][m1]
                    if weight > best :
                        best = weight
                        bestW = m1
            neighbors.append(bestW)

        # PREDICT the rating by user using the user's ratings for the K neighbors

        # if movie has no similar ratings
        if len(neighbors) == 0:
            # print(movie, user)
            pred = RAT[movie]

        else:
            # weighted rating for m1 from user
            rSum = 0

            # sum of all weights
            wSum = 0

            for m1 in neighbors:
                rSum += (ITM[m1][user] * SIM[movie][m1])
                wSum += abs(SIM[movie][m1])
            pred = round((rSum / wSum), 2)

            if pred < 0:
                pred = 0.0
            if pred > 5:
                pred = 5.0
                
            trueRatings2.append(truerating)
            predRatings2.append(pred)
            
        predRatings.append(pred)
            
            
            

    # Compute the Mean-Squared Error between the true and predicted ratings
    mse = round(mean_squared_error(trueRatings, predRatings),4)
    mse2 = round(mean_squared_error(trueRatings2, predRatings2),4)
    output += "\n{}\t{}\t\t{}".format(k, mse, mse2)
    #print('Mean-Squared-Error for k =',k,':', mse)
    #print('Mean-Squared-Error for k =',k,':', mse2)
    ifile.close()
print(output)

Mean Squared Error
K	Average Case	Ignored Case
1	1.4715		1.9859
2	1.4189		1.8413
3	1.393		1.7701
4	1.3941		1.7732
5	1.3911		1.765
5	1.3911		1.765
6	1.3812		1.7377
7	1.3864		1.752
8	1.3848		1.7476
9	1.3828		1.7421
10	1.3838		1.745


# Results

Mean-Squared-Error seems to converge to a common value as k increases. I believe this happens for 2 reasons. 

1. As k increases the neighbors that are chosen for a particular item will have lesser weight therefore not affecting the prediction as much as the neighbors that are closer. 

2. There is a total amount of possible neighbors (n) for any particular movie. If k is bigger than the total amount of neighbors for this one movie, then this movie will be treated as only having n neighbors instead of k neighbors. For example if all movies have total neighbors less than 10, then for any value of k > 10. The algorithm will only pick the total amount of neighbors so the prediction will be very similar for values of k >= 10

When test case has no neighbors then we can not choose any weights for prediction so there are two methods for getting around this
+ Average Case - setting the prediction to the movie's average rating
+ Ignored Case - ignoring the whole test case and not giving a prediction

When comparing the Average case vs the Ignored Case, the error for average case is lower because we are basically just setting the prediction to the average of what that movie rating is. Since the average rating can be seen as the expected and most seen value, then it is a safe prediction when we have nothing else to predict from. 